In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import wandb
wandb.login()

wandb: Currently logged in as: sahaana (use `wandb login --relogin` to force relogin)


True

# Imports 

In [5]:
import pandas as pd
import numpy as np
from collections import defaultdict


import matplotlib.pyplot as plt
from transformers import AutoTokenizer, DistilBertModel

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

In [6]:
import sys
#sys.path.append('/lfs/1/sahaana/enrichment/enrich/utils')
sys.path.append('/lfs/1/sahaana/enrichment/ember/utils')
 
    
from embedding_datasets import MARCODataset, EmberEvalDataset
from embedding_models import TripletSingleBERTModel
from embedding_utils import param_header, tokenize_batch  
from embedding_runner import train_model, eval_model
#from model_utils import MatchedDatasetTriplets, param_header_bert, tokenize_batch   
#from models import BatchedTripletSingleTowerModel, BatchedTripletSingleBERTModel
#from model_runner import train_model, eval_model
from knn_utils import FaissKNeighbors, knn_MARCO_recall #, knn_matching_accuracy, find_perfect_recall

In [7]:
%load_ext autoreload
%autoreload 2

# Load Processed (cleaned, aligned) Datasets

In [8]:
left = '/lfs/1/sahaana/enrichment/data/MSMARCO/tableA_processed.pkl'
right = '/lfs/1/sahaana/enrichment/data/MSMARCO/tableB_processed.pkl'

left = pd.read_pickle(left).set_index('QID')
right = pd.read_pickle(right).set_index('PID')

# Prepping Data Indices

In [9]:
train_df = '/lfs/1/sahaana/enrichment/data/MSMARCO/qidpidtriples.train.full.2.pkl'
train_df = pd.read_pickle(train_df)#pd.read_csv(train_df, sep="\t", names = ['QID_a', 'PID_p', 'PID_n'])

val_df = '/lfs/1/sahaana/enrichment/data/MSMARCO/qrels.dev.small.pkl'
val_df = pd.read_pickle(val_df)

"""pd.read_csv(val_df, sep='\t',
                     header=None, 
                     usecols=[0,2], 
                     names=['QID', 'PID'])"""

"pd.read_csv(val_df, sep='\t',\n                     header=None, \n                     usecols=[0,2], \n                     names=['QID', 'PID'])"

In [ ]:
train_df.to_pickle('/lfs/1/sahaana/enrichment/data/MSMARCO/qidpidtriples.train.full.2.pkl')
val_df.to_pickle('/lfs/1/sahaana/enrichment/data/MSMARCO/qrels.dev.small.pkl')

## Model H Params

In [10]:
epochs = 1
batch_size = 8
final_size = 200
lr = .00001
tl_margin = 1.0
tl_p = 2
pool_type = "CLS"
column = "merged_all"
shuffle = True
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer_max_length = 512
compute_val = False
train_size = int(len(train_df)/8)
    

bert_path='/lfs/1/sahaana/enrichment/ember/pretraining/models/MARCO-uncased-masked-ALL-BM25'
bert_model = DistilBertModel.from_pretrained(bert_path, return_dict=True)

model_name = f'MARCO-uncased-masked-ALL-BM25'


## DataLoaders

In [ ]:
train_data = DataLoader(MARCODataset(left, right, 250000, column, train_df), 
                        batch_size=batch_size,
                        shuffle = shuffle
                        )

val_data = None 
"""DataLoader(SQuADDataset(left, right, len(val_df), column, val_df), 
                       batch_size=batch_size,
                       shuffle = False
                      )"""

## Init and Run Model

In [ ]:
triplet_loss = nn.TripletMarginLoss(margin=tl_margin, p=tl_p)
losses = []
val_losses = []
model = TripletSingleBERTModel(final_size, pool_type, bert_path)
optimizer = optim.AdamW(model.parameters(), lr=lr)#optim.SGD(model.parameters(), lr=lr)

In [ ]:
save_dir = param_header(batch_size, final_size, lr, pool_type, epochs, train_size)
save_dir = f'models/{model_name}/{save_dir}/'

In [ ]:
wandb.init(project=model_name)

In [ ]:
train_model(model, 
            tokenizer, 
            tokenize_batch, 
            train_data, 
            val_data, 
            triplet_loss, 
            optimizer, 
            epochs, 
            losses, 
            val_losses, 
            save_dir, 
            compute_val, 
            tokenizer_max_length = tokenizer_max_length)

# k-NN Model Performance (100k data)

In [ ]:
left_eval = '/lfs/1/sahaana/enrichment/data/MSMARCO/dev_tableA_processed.pkl'
right_eval = '/lfs/1/sahaana/enrichment/data/MSMARCO/tableB_processed.pkl'
test = '/lfs/1/sahaana/enrichment/data/MSMARCO/qrels.dev.small.pkl'

left_eval = pd.read_pickle(left_eval).set_index('QID')
right_eval = right#pd.read_pickle(right_eval)
test_df = val_df#pd.read_pickle(test) 

In [ ]:
#PATH = "models/single-tower-BERT-triplet-model/batch_size-8-final_size-50-opt_lr-0.0001-tloss_margin-1.0-tloss_p-2-pooling-CLS-shuffle-True00-35-05-01-21"
#model.load_state_dict(torch.load(PATH))

left_eval_data = DataLoader(EmberEvalDataset(left_eval, column, indexed=True), 
                       batch_size=batch_size,
                       shuffle = False
                      )
right_eval_data = DataLoader(EmberEvalDataset(right_eval, column, indexed=True), 
                       batch_size=batch_size,
                       shuffle = False
                      )

In [ ]:
left_index, left_embeddings = eval_model(model, tokenizer, left_eval_data, tokenizer_max_length=512)
right_index, right_embeddings = eval_model(model, tokenizer, right_eval_data, tokenizer_max_length=512)

In [ ]:
knn = FaissKNeighbors(k=30)
knn.fit(right_embeddings)
neib = knn.kneighbors(left_embeddings)

In [ ]:
knn_results = defaultdict(list)
for k in range(1,31):
    avg, count, MRR, results, MRR_results = knn_MARCO_recall(neib[0], neib[1], test_df, left_index, right_index, k=k)
    knn_results['k'].append(k)
    knn_results['avg'].append(avg)
    knn_results['count'].append(count)
    knn_results['MRR'].append(MRR)
    knn_results['results'].append(results)
    knn_results['MRR_results'].append(MRR_results)

In [ ]:
pd.DataFrame(knn_results)

# k-NN Model Performance (37982 batches)

In [16]:
left_eval = '/lfs/1/sahaana/enrichment/data/MSMARCO/dev_tableA_processed.pkl'
right_eval = '/lfs/1/sahaana/enrichment/data/MSMARCO/tableB_processed.pkl'
test = '/lfs/1/sahaana/enrichment/data/MSMARCO/qrels.dev.small.pkl'

left_eval = pd.read_pickle(left_eval).set_index('QID')
right_eval = right#pd.read_pickle(right_eval)
test_df = val_df#pd.read_pickle(test) 

In [17]:
#PATH = "models/single-tower-BERT-triplet-model/batch_size-8-final_size-50-opt_lr-0.0001-tloss_margin-1.0-tloss_p-2-pooling-CLS-shuffle-True00-35-05-01-21"
#model.load_state_dict(torch.load(PATH))

left_eval_data = DataLoader(EmberEvalDataset(left_eval, column, indexed=True), 
                       batch_size=batch_size,
                       shuffle = False
                      )
right_eval_data = DataLoader(EmberEvalDataset(right_eval, column, indexed=True), 
                       batch_size=batch_size,
                       shuffle = False
                      )

In [18]:
left_index, left_embeddings = eval_model(model, tokenizer, left_eval_data, tokenizer_max_length=512)
right_index, right_embeddings = eval_model(model, tokenizer, right_eval_data, tokenizer_max_length=512)

In [19]:
knn = FaissKNeighbors(k=30)
knn.fit(right_embeddings)
neib = knn.kneighbors(left_embeddings)

In [20]:
knn_results = defaultdict(list)
for k in range(1,31):
    avg, count, MRR, results, MRR_results = knn_MARCO_recall(neib[0], neib[1], test_df, left_index, right_index, k=k)
    knn_results['k'].append(k)
    knn_results['avg'].append(avg)
    knn_results['count'].append(count)
    knn_results['MRR'].append(MRR)
    knn_results['results'].append(results)
    knn_results['MRR_results'].append(MRR_results)

In [21]:
pd.DataFrame(knn_results)

,k,avg,count,MRR,results,MRR_results
0,1,0.149140,1041,0.149140,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
1,2,0.227221,1586,0.188181,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
2,3,0.277650,1938,0.204990,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, ..."
3,4,0.314470,2195,0.214195,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 1.0,..."
4,5,0.340831,2379,0.219468,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 1.0,..."
5,6,0.366476,2558,0.223742,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 1.0,..."
6,7,0.382665,2671,0.226054,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 1.0,..."
7,8,0.397421,2774,0.227899,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 1.0,..."
8,9,0.411318,2871,0.229443,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 1.0,..."
9,10,0.423926,2959,0.230704,"[0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 1.0,..."
